In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers import BatchNormalization
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [7]:
train_df = pd.read_csv("/kaggle/input/heartbeat/mitbih_train.csv", header = None)
test_df = pd.read_csv("/kaggle/input/heartbeat/mitbih_test.csv", header = None)

In [8]:
train_df.shape

There are five categories in the dataset - normal, supraventricular ectopic, ventricular ectopic, fusion and unknown

In [9]:
X_train=train_df.iloc[:,:186].values
X_test=test_df.iloc[:,:186].values
X_train = X_train.reshape(len(X_train), X_train.shape[1],1)
X_test = X_test.reshape(len(X_test), X_test.shape[1],1)
y_train=train_df[187]
y_test=test_df[187]
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [16]:
im_shape=(X_train.shape[1],1)
inputs=Input(shape=(im_shape), name='inputs_cnn')
conv1=Convolution1D(64, (6), activation='relu', input_shape=im_shape)(inputs)
conv1=BatchNormalization()(conv1)
pool1=MaxPool1D(pool_size=(3), strides=(2), padding="same")(conv1)
conv2=Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool1)
conv2=BatchNormalization()(conv2)
pool2=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv2_1)
conv3=Convolution1D(64, (3), activation='relu', input_shape=im_shape)(pool2)
conv3=BatchNormalization()(conv3_1)
pool3=MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv3)
flatten=Flatten()(pool3)
dense1 = Dense(64, activation='relu')(flatten)
dense2 = Dense(32, activation='relu')(dense1)
main_output = Dense(5, activation='softmax', name='main_output')(dense2)

In [17]:
model = Model(inputs= inputs_cnn, outputs=main_output)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])


callbacks = [EarlyStopping(monitor='val_loss', patience=8),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [18]:
history=model.fit(X_train, y_train,epochs=5,callbacks=callbacks, batch_size=32,validation_data=(X_test,y_test))


In [19]:
model.save('/kaggle/working/model.h5')

In [20]:
model.evaluate((X_test),y_test, verbose=0)